In [23]:
import pandas as pd
import os

In [ ]:
df_bio = pd.read_pickle('../data_raw/df_bio.pkl')
df_condition = pd.read_pickle('../data_raw/df_condition.pkl')
df_dedup_deterministic = pd.read_pickle('../data_raw/df_dedup_deterministic.pkl')
df_dedup_proba = pd.read_pickle('../data_raw/df_dedup_proba.pkl')
df_note = pd.read_pickle('../data_raw/df_note.pkl')
df_person = pd.read_pickle('../data_raw/df_person.pkl')
df_visit = pd.read_pickle('../data_raw/df_visit.pkl')

**df_person**

In [ ]:
# Le CDM est identique pour tous les patients, on le supprime
assert(len(df_person['cdm_source'].unique()) == 1)
df_person_clean = df_person.drop(columns=['cdm_source'])

# On supprime les patients sans date de naissance, ou avec une date de naissance trop ancienne (incoherent)
df_person_clean = df_person_clean[df_person_clean['birth_datetime'] > '1920-01-01']

# On adopte une unique convention pour le genre
df_person_clean['gender_source_value'] = df_person_clean['gender_source_value'].replace({'f': 'female', 'm': 'male'})
assert(len(df_person_clean.gender_source_value.unique()) == 2)

# On supprime les hommes de l'étude
df_person_clean = df_person_clean[df_person_clean['gender_source_value'] == 'female']
assert(len(df_person_clean.gender_source_value.unique()) == 1)

# On sauvegarde les données nettoyées dans un répertoire spécifique
output_dir = '../data_clean'
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'df_person.pkl')
df_person_clean.to_pickle(output_path)

**df_bio**

In [56]:
df_bio_clean = df_bio

output_path = os.path.join(output_dir, 'df_bio.pkl')
df_bio_clean.to_pickle(output_path)

**df_visit**

In [55]:
df_visit_clean = df_visit[df_visit['person_id'].isin(df_person_clean['person_id'])]

output_path = os.path.join(output_dir, 'df_visit.pkl')
df_visit_clean.to_pickle(output_path)

**df_note**

In [65]:
df_note_clean = df_note.drop(columns=['cdm_source'])
df_note_clean = df_note_clean[df_note_clean['visit_occurrence_id'].isin(df_visit_clean['visit_occurrence_id'])]

output_path = os.path.join(output_dir, 'df_note.pkl')
df_note_clean.to_pickle(output_path)


In [66]:
df_condition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   visit_occurrence_id      2152 non-null   float64
 1   person_id                2152 non-null   float64
 2   condition_occurrence_id  2152 non-null   float64
 3   condition_source_value   2152 non-null   object 
dtypes: float64(3), object(1)
memory usage: 67.4+ KB


In [ ]:
df_condition.head()